## Imports

In [1]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Load the data

In [4]:
# The parsed html listing data is the interim directory
listing_df = pd.read_csv('/Users/pandabear/springboard/CapstoneTwoProject/data/interim/listing_df.csv',index_col='listing_id')

In [5]:
listing_df.head()

,listing_address,listing_bedbath,listing_body,listing_date,listing_first_image,listing_info,listing_price,listing_sqft,listing_title,listing_url
listing_id,,,,,,,,,,
post id: 7517797369,1537 Jones St near Pacific,1BR / 1Ba,QR Code Link to This Post This is a 1BR 1B...,2022-08-05T13:48:22-0700,https://images.craigslist.org/00X0X_aCkV5CnbZ2...,apartment laundry in bldg attached garage ...,2375,NaN,"Cozy 1BR 1BA, Clean Unit, Walk to Chinatown/Fi...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7519257988,1290 20th ave.,1BR / 1Ba,QR Code Link to This Post 1290 20th Ave. #...,2022-08-09T09:19:13-0700,https://images.craigslist.org/00707_34DEnZksvZ...,application fee details: $35 cats are OK - p...,2995,NaN,1290 20thAve.&Irving/Awesome views/Available N...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7515346170,1825 Mission Street near 14th street.,0BR / 1Ba,QR Code Link to This Post Welcome to 1825 ...,2022-07-30T12:42:45-0700,https://images.craigslist.org/00y0y_gIofOzLqJq...,cats are OK - purrr dogs are OK - wooof apa...,1950,274.0,"Beautiful Landscaping, Stainless Steel Applian...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7514598032,van ness near greenwich street,0BR / 1Ba,QR Code Link to This Post MORE THAN JUST A...,2022-07-28T15:10:30-0700,https://images.craigslist.org/00101_61nNL8QiJS...,cats are OK - purrr dogs are OK - wooof apa...,2850,580.0,Deluxe Bright Studio in The Marina Chateau,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7517784826,780 Post street,0BR / 1Ba,"QR Code Link to This Post 780 Post Street,...",2022-08-05T13:21:42-0700,https://images.craigslist.org/00M0M_cGnsXvY7FB...,cats are OK - purrr dogs are OK - wooof apa...,1575,NaN,Studio in Historic Pet-friendly Building! 780 ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...


In [6]:
listing_df.info

<bound method DataFrame.info of                                                listing_address  \
listing_id                                                       
post id: 7517797369                 1537 Jones St near Pacific   
post id: 7519257988                             1290 20th ave.   
post id: 7515346170      1825 Mission Street near 14th street.   
post id: 7514598032             van ness near greenwich street   
post id: 7517784826                            780 Post street   
post id: 7510575343             Taylor Street near Clay Street   
post id: 7508771898  50 Jones Street near Market & Golden Gate   
post id: 7516882839                           1395 22nd Street   
post id: 7518971104                     Jones Street near Clay   
post id: 7516184254                         franklin near turk   
post id: 7508786938                        200 Buchanan Street   
post id: 7514590718                              900 Folsom St   
post id: 7511563431               449 9th St

In [26]:
listing_df.shape

(6211, 10)

## Exploring the data

In [30]:
# Look at the number of exact duplicates
duplicates = listing_df['listing_title'].value_counts()
duplicates.head()

First-floor two-bedroom in Theater District                               146
Boutique Residential, Parking Available, Prime Location, Pets Welcomed     27
Unparalleled Service, Unbeatable Value                                     21
Walnut Creek! New property near BART full size 3 story gym, appliances     11
Nob Hill Two Bedroom One Bath with Garage Parking ~ J.Wavro                11
Name: listing_title, dtype: int64

In [31]:
print('Total number of duplicates by listing title: {}'.format(duplicates[duplicates != 1].sum()))

Total number of duplicates by listing title: 2012


In [ ]:
# Concat dfs from different scrapes i.e. different directories into a single df before cleaning

In [ ]:
# Convert the columns into the correct dtype
# Convert listing_info into categorical variables
# Clean up listing_body
# Split listing_bedbath into 2 columns
# Cross reference address to make sure it is in San Francisco, otherwise remove from df
